# Resumen Capítulo 6: Bases de Datos Relacionales con SQLite

## 1. Introducción a las Bases de Datos Relacionales

### 1.1 Conceptos Fundamentales

**Base de datos**: Colección de datos que describe las actividades de una o más organizaciones relacionadas.

**DBMS (Data Base Management System)**: Software diseñado para ayudar a mantener y utilizar grandes volúmenes de datos.

**Modelo de datos**: Colección de constructos de descripción de datos de alto nivel que ocultan detalles de almacenamiento de bajo nivel.

**Ventajas de usar un DBMS**:
- Simplifica la administración de datos
- Permite extraer información útil de manera oportuna
- Evita que los datos se conviertan en un pasivo
- Facilita el mantenimiento y uso de grandes volúmenes de información


### 1.2 El Modelo Relacional

El modelo relacional es simple y elegante:
- Una **base de datos** es una colección de una o más **relaciones**
- Cada **relación** es una **tabla** con filas y columnas
- Permite comprender fácilmente el contenido
- Permite el uso de lenguajes simples de alto nivel para consultar los datos

**Componentes de una relación**:
- **Esquema de relación** (*relation schema*): Especifica el nombre de la relación, nombre de cada campo y dominio (tipo de dato)
- **Instancia**: Conjunto de tuplas (registros/filas) que cumplen con el esquema

**Ejemplo de esquema**:
```
Estudiantes(id: STRING, nombre: STRING, login: STRING, edad: INTEGER, promedio: REAL)
```

**Dominio**: Tipo de dato que restringe los valores que pueden aparecer en una columna (STRING, INTEGER, REAL, DATE, etc.)


## 2. Restricciones de Integridad

Las restricciones de integridad son condiciones especificadas en el esquema que restringen los datos que se pueden almacenar.

### 2.1 Restricciones de Dominio

Especifican el tipo de dato y posibles valores permitidos para cada columna.

### 2.2 Restricciones de Llave

**Llave candidata**: Conjunto mínimo de campos que identifica de forma única una tupla.

**Características**:
- Dos tuplas distintas no pueden tener valores idénticos en todos los campos de una llave
- Ningún subconjunto del conjunto de campos en una llave es un identificador único

**Llave primaria (PK)**: Una de las llaves candidatas seleccionada como principal. Se usa para referenciar tuplas desde otras tablas.

**Superllave**: Conjunto de campos que contiene una llave (puede tener campos adicionales).


### 2.3 Restricciones de Llave Foránea (FK)

**Llave foránea**: Campo o conjunto de campos en una relación que hace referencia a la llave primaria de otra relación.

**Características**:
- Debe coincidir en número de columnas y tipos de dato con la llave primaria referenciada
- Los nombres de las columnas pueden ser diferentes
- Puede referirse a la misma relación (auto-referencia)
- Puede contener valores NULL (no viola la restricción)

**Ejemplo**:
```
Inscrito(id_estudiante: STRING, id_curso: STRING, nota: REAL)
  - id_estudiante es FK → Estudiantes(id)
  - id_curso es FK → Curso(id)
```


## 3. Structured Query Language (SQL)

SQL es el lenguaje de bases de datos relacionales más ampliamente utilizado.

### 3.1 Componentes de SQL

**DDL (Data Definition Language)**: Lenguaje de definición de datos
- CREATE TABLE: Crear tablas
- ALTER TABLE: Modificar estructura de tablas
- DROP TABLE: Eliminar tablas

**DML (Data Manipulation Language)**: Lenguaje de manipulación de datos
- SELECT: Consultar datos
- INSERT: Insertar datos
- UPDATE: Actualizar datos
- DELETE: Eliminar datos


### 3.2 DDL - Creación de Tablas

**Sintaxis básica**:
```sql
CREATE TABLE nombre_tabla (
    columna1 TIPO restricciones,
    columna2 TIPO restricciones,
    PRIMARY KEY (columna1),
    FOREIGN KEY (columna2) REFERENCES otra_tabla(columna)
);
```

**Tipos de datos comunes**:
- INTEGER: Números enteros
- REAL: Números decimales
- TEXT: Cadenas de caracteres
- DATE: Fechas

**Restricciones comunes**:
- PRIMARY KEY: Llave primaria
- FOREIGN KEY: Llave foránea
- NOT NULL: Campo obligatorio
- UNIQUE: Valores únicos
- CHECK: Validación de valores


In [ ]:
# Ejemplo de creación de tabla
import sqlite3

connection = sqlite3.connect('ejemplo.db')
cursor = connection.cursor()

cursor.execute("""
CREATE TABLE Estudiantes (
    id TEXT PRIMARY KEY,
    nombre TEXT NOT NULL,
    edad INTEGER CHECK(edad >= 18),
    promedio REAL
);
""")

connection.commit()
connection.close()


## 4. DML - Consultas SQL

### 4.1 Estructura Básica de una Consulta

```sql
SELECT [DISTINCT] columnas
FROM tablas
WHERE condiciones
ORDER BY columnas [ASC|DESC]
```

**Componentes**:
- **SELECT**: Especifica las columnas a mostrar
- **FROM**: Especifica las tablas involucradas
- **WHERE**: Filtra las filas según condiciones
- **DISTINCT**: Elimina duplicados
- **ORDER BY**: Ordena los resultados


### 4.2 Consultas entre Tablas (JOIN)

**JOIN**: Combina filas de dos o más tablas basándose en una condición relacionada.

**Tipos de JOIN**:
- **INNER JOIN**: Solo filas que coinciden en ambas tablas
- **LEFT JOIN**: Todas las filas de la tabla izquierda + coincidencias
- **RIGHT JOIN**: Todas las filas de la tabla derecha + coincidencias
- **FULL OUTER JOIN**: Todas las filas de ambas tablas

**Sintaxis implícita (producto cartesiano)**:
```sql
SELECT E.nombre, C.nombre
FROM Estudiantes E, Cursos C, Inscrito I
WHERE E.id = I.id_estudiante AND C.id = I.id_curso
```

**Sintaxis explícita**:
```sql
SELECT E.nombre, C.nombre
FROM Estudiantes E
JOIN Inscrito I ON E.id = I.id_estudiante
JOIN Cursos C ON C.id = I.id_curso
```


### 4.3 Matching de Patrones en Strings

**LIKE**: Permite buscar patrones en cadenas de texto.

**Comodines**:
- `%`: Cualquier secuencia de caracteres (incluyendo ninguno)
- `_`: Un solo carácter

**Ejemplos**:
```sql
SELECT * FROM Estudiantes WHERE nombre LIKE 'MA%'  -- Nombres que empiezan con MA
SELECT * FROM Estudiantes WHERE nombre LIKE '%ez'  -- Nombres que terminan con ez
SELECT * FROM Estudiantes WHERE nombre LIKE '_a%'  -- Nombres con 'a' en segunda posición
```


### 4.4 Operaciones de Conjuntos

**UNION**: Combina resultados de dos consultas (elimina duplicados)
```sql
SELECT id FROM Estudiantes WHERE promedio > 5.0
UNION
SELECT id_estudiante FROM Inscrito WHERE id_curso = 'IIC2115'
```

**INTERSECT**: Intersección (elementos en ambas consultas)

**EXCEPT (MINUS)**: Diferencia (elementos en primera pero no en segunda)

**Operadores de comparación de conjuntos**:
- **IN**: Verifica si un valor está en un conjunto
- **EXISTS**: Verifica si un conjunto tiene elementos
- **ANY/ALL**: Compara un valor con elementos de un conjunto


### 4.5 Consultas Anidadas (Subconsultas)

Una consulta anidada tiene otra consulta incrustada dentro de ella.

**Ejemplo con NOT IN**:
```sql
SELECT nombre
FROM Estudiantes E
WHERE E.id NOT IN (
    SELECT id_estudiante
    FROM Inscrito
    WHERE id_curso = 'IIC2115'
)
```

**Ejemplo con EXISTS**:
```sql
SELECT nombre
FROM Estudiantes E
WHERE EXISTS (
    SELECT *
    FROM Inscrito I
    WHERE I.id_curso = 'IIC2115' AND I.id_estudiante = E.id
)
```

**Ejemplo con ALL**:
```sql
SELECT nombre
FROM Estudiantes E
WHERE E.promedio > ALL (
    SELECT promedio
    FROM Estudiantes E2
    WHERE E2.nombre = 'José'
)
```


### 4.6 Agregación

**Funciones de agregación**:
- **COUNT([DISTINCT] columna)**: Cantidad de valores
- **SUM([DISTINCT] columna)**: Suma de valores
- **AVG([DISTINCT] columna)**: Promedio de valores
- **MAX(columna)**: Valor máximo
- **MIN(columna)**: Valor mínimo

**Ejemplo**:
```sql
SELECT AVG(edad) as edad_promedio
FROM Estudiantes
WHERE promedio > 5.0
```


### 4.7 Agrupación (GROUP BY y HAVING)

**GROUP BY**: Agrupa filas que tienen los mismos valores en columnas especificadas.

**HAVING**: Filtra grupos después de la agregación (similar a WHERE pero para grupos).

**Ejemplo**:
```sql
SELECT promedio, AVG(edad) as edad_promedio
FROM Estudiantes
GROUP BY promedio
HAVING promedio >= 5
```

**Diferencia WHERE vs HAVING**:
- **WHERE**: Filtra filas antes de la agregación
- **HAVING**: Filtra grupos después de la agregación


## 5. Acceso a Bases de Datos con Python (SQLite)

### 5.1 Introducción a SQLite

SQLite es un administrador de bases de datos:
- Altamente compacto y funcional
- Compatible con SQL
- Disponible para Windows, Linux y Mac
- Ideal para aplicaciones académicas y pequeñas
- Las bases de datos se almacenan en un archivo

### 5.2 Conexión y Operaciones Básicas

**Módulo**: `sqlite3` (incluido en Python)


In [ ]:
import sqlite3

# Crear o abrir una base de datos
connection = sqlite3.connect('ejemplo.db')

# Crear un cursor para ejecutar comandos
cursor = connection.cursor()

# Ejecutar comandos SQL
cursor.execute("CREATE TABLE countries(name TEXT, continent TEXT, population INTEGER)")
cursor.execute("INSERT INTO countries VALUES ('Chile', 'America', 18006407)")

# Hacer commit para guardar cambios
connection.commit()

# Cerrar la conexión
connection.close()


### 5.3 Inserción Múltiple (executemany)

Permite insertar múltiples valores de manera eficiente:


In [ ]:
countries = [
    ('Jamaica', 'America', 2881355),
    ('Chile', 'America', 18006407),
    ('Australia', 'Oceania', 24680100)
]

cursor.executemany('INSERT INTO countries VALUES (?,?,?)', countries)
connection.commit()


### 5.4 Consultas y Obtención de Resultados

**Métodos para obtener resultados**:
- `fetchone()`: Obtiene una fila
- `fetchall()`: Obtiene todas las filas
- Iterar sobre el cursor: `for row in cursor.execute(...)`


In [ ]:
# Consulta simple
cursor.execute('SELECT * FROM countries')
resultado = cursor.fetchall()
print(resultado)

# Iterar sobre resultados
for country in cursor.execute('SELECT * FROM countries ORDER BY name'):
    print(country)


### 5.5 Parametrización Segura de Consultas

**IMPORTANTE**: Siempre usar sustitución de parámetros para evitar **SQL Injection**.

**Método seguro (usar ?)**:
```python
country = ('Chile',)
cursor.execute('SELECT * FROM countries WHERE name = ?', country)
```

**Método inseguro (NO usar)**:
```python
country = "'Chile'; DROP TABLE countries;"
cursor.execute(f"SELECT * FROM countries WHERE name = {country}")  # PELIGROSO
```


## 6. Proceso de Creación de una Base de Datos

### 6.1 Pasos Generales

1. **Exploración inicial de datos**: Analizar archivos CSV/JSON para entender la estructura
2. **Modelación de entidades**: Identificar entidades principales y sus atributos
3. **Diseño relacional**: Definir tablas, llaves primarias y foráneas
4. **Creación de esquema**: Implementar tablas con CREATE TABLE
5. **Carga de datos**: Insertar datos desde archivos a las tablas
6. **Verificación**: Comprobar que los datos se cargaron correctamente


## 7. Conceptos Clave para Recordar

### 7.1 Restricciones de Integridad

| Tipo | Descripción | Ejemplo |
|------|-------------|---------|
| Dominio | Tipo de dato permitido | INTEGER, TEXT, REAL |
| Llave Primaria | Identificador único | PRIMARY KEY (id) |
| Llave Foránea | Referencia a otra tabla | FOREIGN KEY (id_estudiante) REFERENCES Estudiantes(id) |
| NOT NULL | Campo obligatorio | nombre TEXT NOT NULL |
| UNIQUE | Valores únicos | email TEXT UNIQUE |
| CHECK | Validación de valores | edad INTEGER CHECK(edad >= 18) |

### 7.2 Operaciones SQL Comunes

| Operación | Sintaxis | Propósito |
|-----------|----------|-----------|
| SELECT | SELECT columnas FROM tabla | Consultar datos |
| INSERT | INSERT INTO tabla VALUES (...) | Insertar datos |
| UPDATE | UPDATE tabla SET columna=valor WHERE condición | Actualizar datos |
| DELETE | DELETE FROM tabla WHERE condición | Eliminar datos |
| JOIN | SELECT ... FROM tabla1 JOIN tabla2 ON condición | Combinar tablas |
| GROUP BY | SELECT ... GROUP BY columna | Agrupar resultados |
| HAVING | SELECT ... GROUP BY ... HAVING condición | Filtrar grupos |

### 7.3 Buenas Prácticas

1. **Siempre usar parámetros** (`?`) en lugar de concatenación de strings
2. **Hacer commit** después de modificaciones
3. **Cerrar conexiones** cuando no se necesiten
4. **Validar datos** antes de insertar
5. **Usar transacciones** para operaciones complejas
6. **Crear índices** en columnas frecuentemente consultadas
7. **Normalizar** el diseño de la base de datos
